In [1]:
import librenderman as rm
import numpy as np
import gzip
import cPickle
import os
import scipy
import librosa
from IPython.display import Audio

In [2]:
nsynth = cPickle.load(gzip.open('Data/NSynth.data', 'rb'))

In [9]:
synth_options = ['Dexed', 'Obxd', 'Synth1', 'TAL-NoiseMaker', 'Tunefish4', 'ComboF']
synth = synth_options[2]

#results = cPickle.load(gzip.open('Results/' + synth + '_LSTM_NSynth.data', 'rb'))
results = cPickle.load(open('Results/' + synth + '_LSTM_NSynth.data', 'rb'))

                                 
sampleRate = 44100
bufferSize = 512
fftSize = 512

nmfccs = 6

noteLength = 1.0
renderLength = 1.5
midiNote = 60
midiVelocity = 127

autosynth_dir = os.path.abspath(os.path.join(os.environ['PWD'], os.pardir))

samples = len(results)

engine = rm.RenderEngine(sampleRate, bufferSize, fftSize)

path = autosynth_dir + "/Synths/" + synth + '.vst'
engine.load_plugin(path)

generator = rm.PatchGenerator(engine)

In [4]:
total_dist = 0
for i in range(samples):
    test = results[i]
    test_patch = []
    for i in range(engine.get_plugin_parameter_size()):
        if test[i] >= 0:
            test_patch.append((i, float(test[i])))
        else:
            test_patch.append((i, 0))
    if synth is 'Dexed':
            overridden_parameters = [(26, 1.),  (30, 0.),  (48, 1.),  (52, 0.), 
                        (70, 1.),  (74, 0.),  (92, 1.),  (96, 0.), 
                        (114, 1.), (118, 0.), (136, 1.), (140, 0.)]

    elif synth is 'Obxd':
            overridden_parameters = [(1, 0.),  (70, 0.)]
            
    elif synth is 'ComboF':
            overridden_parameters = [(19, 1.),  (20, 0.), (21, 0.),  (22, 0.),
                                 (23, 0.),  (24, 0.), (25, 0.),  (26, 0.),
                                 (27, 0.),  (28, 0.), (29, 0.),  (30, 0.),
                                 (34, 0.)]
    elif synth is 'TAL-NoiseMaker':
            test_patch.append(91,0.5)
    
    true_audio = nsynth[i]
    
    engine.set_patch(test_patch)

    for parameter in overridden_parameters:
        index, value = parameter
        engine.override_plugin_parameter(index, value)
    
    engine.render_patch(midiNote, midiVelocity, noteLength, renderLength)
    
    test_audio = engine.get_audio_frames()
    
    true_mfccs = librosa.feature.mfcc(y=np.array(np.append(true_audio, np.zeros(500))), sr=sampleRate,n_mfcc=nmfccs)
    test_mfccs = librosa.feature.mfcc(y=np.array(test_audio), sr=sampleRate,n_mfcc=nmfccs)
    
    true_mfccs = [item for sublist in true_mfccs for item in sublist]
    test_mfccs = [item for sublist in test_mfccs for item in sublist]
    
    distance = scipy.spatial.distance.euclidean(true_mfccs, test_mfccs)
    
    total_dist += distance
    
avg_dist = total_dist/float(samples)

IndexError: index 42 is out of bounds for axis 0 with size 42

In [ ]:
avg_dist

In [58]:
Audio(test_audio, rate=sampleRate)

In [59]:
Audio(true_audio, rate=sampleRate)

In [10]:
obxd_test = test_audio
Audio(obxd_test, rate=sampleRate)

In [11]:
obxd_true = true_audio
Audio(obxd_true, rate=sampleRate)